In [1]:
# Import necessary libraries
import os
from typing import Any, Dict

import google.generativeai as genai
import requests
from dotenv import load_dotenv

print("We are up and running!")

We are up and running!


Ensure that you created a Gemini API Key and added it to the `.env` file before executing the next command

In [5]:
# Load environment variables
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")
if not api_key or api_key.startswith("ADD YOUR"):
    raise ValueError("GEMINI_API_KEY not found in .env file")

# Configure Gemini
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash") # Changed to 2.5!

print(f"Gemini model loaded successfully: {model.model_name}")

Gemini model loaded successfully: models/gemini-2.5-flash


In [6]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT)
print(response.text)

I cannot provide the *exact, real-time current* price of Bitcoin because I am an AI and my knowledge is based on my last training update, not live data. Bitcoin's price fluctuates constantly.

To get the most up-to-date price, you should check:

*   **Cryptocurrency Exchanges:** Binance, Coinbase, Kraken, etc.
*   **Financial News Websites:** Reuters, Bloomberg, CoinDesk, Cointelegraph.
*   **Crypto Price Trackers:** CoinMarketCap, CoinGecko.
*   **A quick Google search** for "Bitcoin price" will usually show a live quote from a reliable source.


In [7]:
print(response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "I cannot provide the *exact, real-time current* price of Bitcoin because I am an AI and my knowledge is based on my last training update, not live data. Bitcoin's price fluctuates constantly.\n\nTo get the most up-to-date price, you should check:\n\n*   **Cryptocurrency Exchanges:** Binance, Coinbase, Kraken, etc.\n*   **Financial News Websites:** Reuters, Bloomberg, CoinDesk, Cointelegraph.\n*   **Crypto Price Trackers:** CoinMarketCap, CoinGecko.\n*   **A quick Google search** for \"Bitcoin price\" will usually show a live quote from a reliable source."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 9,


In [ ]:
url = f"https://api.binance.com/api/v3/ticker/price?symbol=BTCUSDT"
response = requests.get(url)
data = response.json()
print(data) # Given the error, the next line just hardcodes the price
data = {'symbol': 'BTCUSDT', 'price': '84399.37'} # Mocked response for testing


{'code': 0, 'msg': "Service unavailable from a restricted location according to 'b. Eligibility' in https://www.binance.com/en/terms. Please contact customer service if you believe you received this message in error."}


In [10]:

# Define the function
def get_crypto_price(symbol: str) -> Dict[str, Any]:
    """
    Get the current price of a cryptocurrency from Binance API
    """
    url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
    response = requests.get(url)
    data = response.json()
    data = {'symbol': 'BTCUSDT', 'price': '84399.37'} # Mocked response for testing

    return float(data["price"])


In [12]:
price = get_crypto_price("BTCUSDT")
print(f"BTC Price in USDT: {price} ({type(price)})")

BTC Price in USDT: 84399.37 (<class 'float'>)


In [13]:
tools = [
    {
        "function_declarations": [
            {
                "name": "get_crypto_price",
                "description": "Get cryptocurrency price in USDT from Binance",
                "parameters": {
                    "type": "object", 
                    "properties": {
                        "symbol": {
                            "type": "string",
                            "description": "The cryptocurrency trading pair symbol (e.g., BTCUSDT, ETHUSDT). \
                                            The symbol for Bitcoin is BTCUSDT. \
                                            The symbol for Ethereum is ETHUSDT."
                        }
                    },
                    "required": ["symbol"]
                }
            }
        ]
    }
]

In [14]:
PROMPT = "What is the current price of Bitcoin?"
chat = model.start_chat()
response = chat.send_message(PROMPT, tools=tools)
print(response)


response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "function_call": {
                  "name": "get_crypto_price",
                  "args": {
                    "symbol": "BTCUSDT"
                  }
                }
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 89,
        "candidates_token_count": 19,
        "total_token_count": 166
      },
      "model_version": "gemini-2.5-flash"
    }),
)


In [15]:
price = get_crypto_price("BTCUSDT")

In [16]:
final_response = chat.send_message(str(price))
print(final_response)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "The current price of Bitcoin (BTC) is $84,399.37."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 38,
        "candidates_token_count": 20,
        "total_token_count": 58
      },
      "model_version": "gemini-2.5-flash"
    }),
)


In [17]:
print(final_response.text)

The current price of Bitcoin (BTC) is $84,399.37.
